In [84]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

### Read and clean csv

In [111]:
df = pd.read_csv("las2.csv").drop(columns=["Timestamp", "Empty"]).rename(columns={
    "What is your name?": "name",
    "For each of the following, choose which days and times you are available (Please try to put at LEAST 10 available slots)": "times"
}).dropna()
df

,name,times
0,Amy Clark,"Monday 1-2, Monday 2-3, Monday 3-4, Monday 4-5..."
1,Ruchi Sarkar,"Monday 10-11, Monday 1-2, Tues 10-11, Tues 11-..."
2,Dana Rubin,"Monday 10-11, Monday 1-2, Monday 4-5, Tues 1-2..."
3,Nidhi Murlidhar,"Monday 4-5, Tues 10-11, Tues 11-12, Tues 12-1,..."
5,Natasha Vaidya,"Monday 1-2, Monday 2-3, Monday 3-4, Monday 4-5..."
6,Shannon Goad,"Monday 2-3, Monday 3-4, Monday 4-5, Tues 3-4, ..."
7,Halie Chmura,"Monday 11-12, Monday 12-1, Monday 1-2, Monday ..."
8,Sarah Bost,"Monday 10-11, Monday 2-3, Monday 3-4, Monday 4..."
9,Anna Truelove,"Monday 10-11, Monday 11-12, Monday 2-3, Tues 2..."
10,James Bury,"Monday 12-1, Monday 1-2, Monday 2-3, Tues 11-1..."


In [63]:
all_times = set()

for index, row in df.iterrows():
    times = str(row["times"]).split(", ")
    for t in times:
        if t not in all_times:
            all_times.add(t)
            
all_times = list(all_times)

In [64]:
all_times

['Monday 4-5',
 'Tues 11-12',
 'Tues 12-1',
 'Tues 4-5',
 'Thur 3-4',
 'Wed 11-12',
 'Thur 4-5',
 'Monday 10-11',
 'Monday 12-1',
 'Monday 1-2',
 'Thur 2-3',
 'Thur 11-12',
 'Wed 12-1',
 'Monday 2-3',
 'Monday 3-4',
 'Wed 10-11',
 'Wed 2-3',
 'Thur 12-1',
 'Tues 2-3',
 'Wed 3-4',
 'Wed 1-2',
 'Thur 10-11',
 'Wed 4-5',
 'nan',
 'Monday 11-12',
 'Tues 1-2',
 'Thur 1-2',
 'Tues 3-4',
 'Tues 10-11']

### Sort times

In [70]:
days = ["Monday", "Tues", "Wed", "Thur"]
time_order = ["10-11", "11-12", "12-1", "1-2", "2-3", "3-4", "4-5"]
sorted_times = []

for d in days:
    curr_day = [x for x in all_times if d in x]
    for t in time_order:
        for c in curr_day:
            if t in c:
                sorted_times.append(c)
                
print(sorted_times)

['Monday 10-11', 'Monday 11-12', 'Monday 12-1', 'Monday 1-2', 'Monday 2-3', 'Monday 3-4', 'Monday 4-5', 'Tues 10-11', 'Tues 11-12', 'Tues 12-1', 'Tues 1-2', 'Tues 2-3', 'Tues 3-4', 'Tues 4-5', 'Wed 10-11', 'Wed 11-12', 'Wed 12-1', 'Wed 1-2', 'Wed 2-3', 'Wed 3-4', 'Wed 4-5', 'Thur 10-11', 'Thur 11-12', 'Thur 12-1', 'Thur 1-2', 'Thur 2-3', 'Thur 3-4', 'Thur 4-5']


### Create LA and Shift classes

In [128]:
class LA:
    
    def __init__(self, name):
        self.name = name
        self.hours_left = 5
        self.shifts = []
        self.possible_shifts = []
        
    def assign(self, shift):
        self.hours_left -= 1
        self.shifts.append(shift)
        
    def addPossibleShift(self, shift):
        self.possible_shifts.append(shift)
        
    def __repr__(self):
        return "name: " + self.name + ", hours left: " + str(self.hours_left) + ", shifts: " + str([x.title for x in self.shifts])
        
        
class Shift:
    
    def __init__(self, title):
        self.title = title
        self.workers = []
        self.available_workers = []
        self.positions_remaining = 5
        
    def addLAToShift(self, la):
        if self.positions_remaining > 0:
            self.workers.append(la)
            self.positions_remaining -= 1
            return True
        else:
            return False
        
    def addAvailableWorker(self, la):
        self.available_workers.append(la)
        
    def __repr__(self):
        return "Shift: " + self.title + ", LAs assigned: " + str([x.name for x in self.workers]) + \
               ", positions left: " + str(self.positions_remaining) + ", available las: " + \
                str([x.name for x in self.available_workers])
        
l = LA("James")
s = Shift("Monday 10-11")
l.assign(s)
s.addLAToShift(l)
s.addAvailableWorker(l)
print(l)
print(s)

name: James, hours left: 4, shifts: ['Monday 10-11']
Shift: Monday 10-11, LAs assigned: ['James'], positions left: 4, available las: ['James']


### Add time columns

In [132]:
[x for x in shifts if x.title == "Monday 10-11"][0].addAvailableWorker(l)

In [133]:
shifts

[Shift: Monday 10-11, LAs assigned: [], positions left: 5, available las: ['Sophia Shaikh'],
 Shift: Monday 11-12, LAs assigned: [], positions left: 5, available las: [],
 Shift: Monday 12-1, LAs assigned: [], positions left: 5, available las: [],
 Shift: Monday 1-2, LAs assigned: [], positions left: 5, available las: [],
 Shift: Monday 2-3, LAs assigned: [], positions left: 5, available las: [],
 Shift: Monday 3-4, LAs assigned: [], positions left: 5, available las: [],
 Shift: Monday 4-5, LAs assigned: [], positions left: 5, available las: [],
 Shift: Tues 10-11, LAs assigned: [], positions left: 5, available las: [],
 Shift: Tues 11-12, LAs assigned: [], positions left: 5, available las: [],
 Shift: Tues 12-1, LAs assigned: [], positions left: 5, available las: [],
 Shift: Tues 1-2, LAs assigned: [], positions left: 5, available las: [],
 Shift: Tues 2-3, LAs assigned: [], positions left: 5, available las: [],
 Shift: Tues 3-4, LAs assigned: [], positions left: 5, available las: [],

In [134]:
LAs = []
shifts = []

for t in sorted_times:
    df[t] = 0
    shifts.append(Shift(t))
    
for index, row in df.iterrows():
    l = LA(row["name"])
    for t in sorted_times:
        if t in str(row["times"]).split(", "):
            df[t][index] = 1
            l.addPossibleShift(t)
            [x for x in shifts if x.title == t][0].addAvailableWorker(l)
    
    LAs.append(l)
            
print(LAs)
print(shifts)
df.head(2)

[name: Amy Clark, hours left: 5, shifts: [], name: Ruchi Sarkar , hours left: 5, shifts: [], name: Dana Rubin, hours left: 5, shifts: [], name: Nidhi Murlidhar, hours left: 5, shifts: [], name: Natasha Vaidya, hours left: 5, shifts: [], name: Shannon Goad, hours left: 5, shifts: [], name: Halie Chmura, hours left: 5, shifts: [], name: Sarah Bost, hours left: 5, shifts: [], name: Anna Truelove, hours left: 5, shifts: [], name: James Bury, hours left: 5, shifts: [], name: Angel Karafas, hours left: 5, shifts: [], name: Akash Krishna, hours left: 5, shifts: [], name: Sophia Shaikh, hours left: 5, shifts: []]
[Shift: Monday 10-11, LAs assigned: [], positions left: 5, available las: ['Ruchi Sarkar ', 'Dana Rubin', 'Sarah Bost', 'Anna Truelove', 'Angel Karafas'], Shift: Monday 11-12, LAs assigned: [], positions left: 5, available las: ['Halie Chmura', 'Anna Truelove', 'Akash Krishna'], Shift: Monday 12-1, LAs assigned: [], positions left: 5, available las: ['Halie Chmura', 'James Bury'], Shi

,name,times,Monday 10-11,Monday 11-12,Monday 12-1,Monday 1-2,Monday 2-3,Monday 3-4,Monday 4-5,Tues 10-11,...,Wed 2-3,Wed 3-4,Wed 4-5,Thur 10-11,Thur 11-12,Thur 12-1,Thur 1-2,Thur 2-3,Thur 3-4,Thur 4-5
0,Amy Clark,"Monday 1-2, Monday 2-3, Monday 3-4, Monday 4-5...",0,0,0,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0
1,Ruchi Sarkar,"Monday 10-11, Monday 1-2, Tues 10-11, Tues 11-...",1,0,0,1,0,0,0,1,...,0,0,0,1,1,0,0,1,0,0


### Sorts shift times based on LAs available

In [107]:
workers = {}

for t in sorted_times:
    workers[t] = sum(df[t])

# Sort by fewest LAs available
sorted_workers = {k: v for k, v in sorted(workers.items(), key=lambda item: item[1])}
sorted_workers

{'Monday 12-1': 2,
 'Monday 11-12': 3,
 'Wed 11-12': 3,
 'Wed 12-1': 3,
 'Thur 2-3': 3,
 'Thur 3-4': 3,
 'Tues 2-3': 4,
 'Tues 3-4': 4,
 'Thur 4-5': 4,
 'Tues 10-11': 5,
 'Tues 1-2': 5,
 'Tues 4-5': 5,
 'Wed 3-4': 5,
 'Thur 10-11': 5,
 'Thur 12-1': 5,
 'Thur 1-2': 5,
 'Monday 10-11': 6,
 'Monday 3-4': 6,
 'Tues 12-1': 6,
 'Wed 4-5': 6,
 'Monday 1-2': 7,
 'Wed 10-11': 7,
 'Thur 11-12': 7,
 'Monday 2-3': 8,
 'Monday 4-5': 8,
 'Tues 11-12': 8,
 'Wed 1-2': 8,
 'Wed 2-3': 8}

### Assigns shifts

In [ ]:
for t in sorted_times:
    